In [ ]:
import spacy
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import GPT2Model, GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import random

In [ ]:
# Load the SpaCy model (English model in this case)
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model_gpt2 = GPT2Model.from_pretrained("gpt2")

In [ ]:
decoder_model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
# Sample clinical sentences to analyze
sentences = [
    "The black patient was prescribed medication.",
    "The white patient was given a different treatment.",
    "The doctor treated the hispanic patient for a respiratory issue.",
    "The nurse assisted the asian patient during recovery."
]

In [ ]:
# List of racial terms to check for potential bias
racial_terms = ["black", "white", "hispanic", "asian", "indian", "native", "african"]

In [ ]:
# Function to get embeddings from GPT-2
def get_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt")  # Tokenize the input sentence
        with torch.no_grad():
            outputs = model_gpt2(**inputs)  # Get the model output for the input sentence
            embeddings.append(outputs.last_hidden_state.mean(dim=1))  # Get the mean of the hidden states to represent the sentence
    return torch.cat(embeddings, dim=0)  # Concatenate all sentence embeddings

In [ ]:
# Generate positive (biased) and negative (debiased) samples
positive_samples = sentences  # Original biased sentences
negative_samples = [
    sentence.replace(term, "person") for sentence in sentences for term in racial_terms if term in sentence.lower()  # Replace racial terms with neutral terms
]

In [ ]:
positive_samples

In [ ]:
negative_samples

In [ ]:
# Get embeddings for both positive and negative samples
positive_embeddings = get_embeddings(positive_samples)  # Get embeddings for positive samples
negative_embeddings = get_embeddings(negative_samples)  # Get embeddings for negative samples

In [ ]:
# Combine positive and negative samples
data_embeddings = torch.cat((positive_embeddings, negative_embeddings), dim=0)  # Combine positive and negative embeddings
labels = torch.cat((torch.ones(len(positive_samples)), torch.zeros(len(negative_samples))), dim=0)  # Create labels for positive (1) and negative (0) samples

In [ ]:
# Define a simple contrastive learning model
class ContrastiveModel(nn.Module):
    def __init__(self, embedding_dim):
        super(ContrastiveModel, self).__init__()
        self.fc = nn.Linear(embedding_dim, 128)  # Fully connected layer to project to a lower dimension
        self.projection = nn.Linear(128, 64)  # Projection layer for further reducing dimensionality
    
    def forward(self, x):
        x = torch.relu(self.fc(x))  # Apply ReLU activation function to the fully connected layer
        x = self.projection(x)  # Apply projection layer to further reduce dimensionality
        return x

In [ ]:
# Initialize the contrastive model
embedding_dim = data_embeddings.shape[1]  # Get the embedding dimension from the data
contrastive_model = ContrastiveModel(embedding_dim)  # Initialize the contrastive model
criterion = nn.CosineEmbeddingLoss()  # Loss function to measure similarity between positive and negative pairs
optimizer = optim.Adam(contrastive_model.parameters(), lr=0.001)  # Adam optimizer for training the model

In [ ]:
# Train the model using contrastive learning
epochs = 10  # Number of training epochs
batch_size = 2  # Batch size for training
for epoch in range(epochs):
    contrastive_model.train()  # Set the model to training mode
    permutation = torch.randperm(data_embeddings.size(0))  # Shuffle the data
    for i in range(0, data_embeddings.size(0), batch_size):
        indices = permutation[i:i + batch_size]  # Get the indices for the current batch
        batch_embeddings = data_embeddings[indices]  # Get the embeddings for the current batch
        batch_labels = labels[indices]  # Get the labels for the current batch
        
        # Create positive and negative pairs for contrastive learning
        if len(batch_embeddings) < 2:
            continue  # Skip if batch size is less than 2
        emb1, emb2 = batch_embeddings[0], batch_embeddings[1]  # Get two embeddings from the batch
        label = 1 if batch_labels[0] == batch_labels[1] else -1  # Determine if the pair is positive or negative
        
        # Forward pass
        output1 = contrastive_model(emb1)  # Get the output for the first embedding
        output2 = contrastive_model(emb2)  # Get the output for the second embedding
        loss = criterion(output1, output2, torch.tensor(label, dtype=torch.float))  # Compute the loss
        
        # Backward pass and optimization
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model parameters
    
    # Print loss for each epoch
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")  # Print the loss for the current epoch

In [ ]:
# Save the debiased embeddings
contrastive_model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    debiased_embeddings = contrastive_model(data_embeddings)  # Get the debiased embeddings
print("Debiased embeddings generated.")  # Print confirmation that debiased embeddings have been generated

In [ ]:
# Function to reconstruct debiased sentences using the decoder model
def reconstruct_sentences(embeddings, tokenizer, model):
    reconstructed_sentences = []
    for embedding in embeddings:
        # Generate text from the embedding using GPT-2
        input_ids = torch.tensor(tokenizer.encode("[START]", add_special_tokens=True)).unsqueeze(0)  # Encode a special token to start generation
        outputs = model.generate(input_ids=input_ids, max_length=50, num_return_sequences=1)  # Generate a sentence
        reconstructed_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Decode the generated output
        reconstructed_sentences.append(reconstructed_sentence)  # Append the reconstructed sentence to the list
    return reconstructed_sentences  # Return the list of reconstructed sentences

# Reconstruct debiased sentences
reconstructed_sentences = reconstruct_sentences(debiased_embeddings, tokenizer, decoder_model)  # Reconstruct sentences using debiased embeddings

# Print reconstructed sentences
print("Reconstructed Sentences:")  # Print header for reconstructed sentences
for sentence in reconstructed_sentences:
    print(sentence)  # Print each reconstructed sentence


In [ ]:
# Function to reconstruct debiased sentences using the decoder model
def reconstruct_sentences(embeddings, tokenizer, model):
    reconstructed_sentences = []
    for embedding in embeddings:
        # Generate text from the embedding using GPT-2
        # Project the embedding to match GPT-2 input dimensions (if required)
        input_ids = model.generate(input_ids=None, max_length=50, num_return_sequences=1, bos_token_id=tokenizer.bos_token_id, embeddings=embedding.unsqueeze(0))  # Generate a sentence using the debiased embedding
        reconstructed_sentence = tokenizer.decode(input_ids[0], skip_special_tokens=True)  # Decode the generated output
        reconstructed_sentences.append(reconstructed_sentence)  # Append the reconstructed sentence to the list
    return reconstructed_sentences  # Return the list of reconstructed sentences

# Reconstruct debiased sentences
reconstructed_sentences = reconstruct_sentences(debiased_embeddings, tokenizer, decoder_model)  # Reconstruct sentences using debiased embeddings

# Print reconstructed sentences
print("Reconstructed Sentences:")  # Print header for reconstructed sentences
for sentence in reconstructed_sentences:
    print(sentence)  # Print each reconstructed sentence

In [ ]:
def reconstruct_sentences(embeddings, tokenizer, model):
    reconstructed_sentences = []
    for embedding in embeddings:
        # Generate text from the embedding using GPT-2
        inputs_embeds = embedding.unsqueeze(0)  # Adjust the embedding dimension for model input
        attention_mask = torch.ones(inputs_embeds.size()[:-1], dtype=torch.long)  # Create attention mask for the embedding
        outputs = model.generate(inputs_embeds=inputs_embeds, attention_mask=attention_mask, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)  # Generate a sentence
        reconstructed_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Decode the generated output
        reconstructed_sentences.append(reconstructed_sentence)  # Append the reconstructed sentence to the list
    return reconstructed_sentences  # Return the list of reconstructed sentences

# Reconstruct debiased sentences
reconstructed_sentences = reconstruct_sentences(debiased_embeddings, tokenizer, decoder_model)  # Reconstruct sentences using debiased embeddings

# Print reconstructed sentences
print("Reconstructed Sentences:")  # Print header for reconstructed sentences
for sentence in reconstructed_sentences:
    print(sentence)  # Print each reconstructed sentence


In [ ]:
# Function to reconstruct debiased sentences using the decoder model
def reconstruct_sentences(embeddings, tokenizer, model):
    reconstructed_sentences = []
    for embedding in embeddings:
        # Generate text using a prompt initialized with a beginning of sequence token
        input_ids = torch.tensor([[tokenizer.bos_token_id]])  # Create an initial input token
        prompt_embedding = embedding.unsqueeze(0)  # Adjust the embedding dimension for concatenation
        inputs_embeds = torch.cat([model.transformer.wte(input_ids), prompt_embedding], dim=1)  # Concatenate input token embedding with the prompt embedding
        attention_mask = torch.ones(inputs_embeds.size()[:-1], dtype=torch.long)  # Create attention mask
        outputs = model.generate(inputs_embeds=inputs_embeds, attention_mask=attention_mask, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)  # Generate a sentence
        reconstructed_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Decode the generated output
        reconstructed_sentences.append(reconstructed_sentence)  # Append the reconstructed sentence to the list
    return reconstructed_sentences  # Return the list of reconstructed sentences

# Reconstruct debiased sentences
reconstructed_sentences = reconstruct_sentences(debiased_embeddings, tokenizer, decoder_model)  # Reconstruct sentences using debiased embeddings

# Print reconstructed sentences
print("Reconstructed Sentences:")  # Print header for reconstructed sentences
for sentence in reconstructed_sentences:
    print(sentence)  # Print each reconstructed sentence

In [ ]:
RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [ ]:
import spacy
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import GPT2Model, GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import random

# Load the SpaCy model (English model in this case)
try:
    nlp = spacy.load("en_core_web_sm")  # Attempt to load the pre-trained SpaCy model
except OSError:
    import spacy.cli
    spacy.cli.download("en_core_web_sm")  # Download the model if not found
    nlp = spacy.load("en_core_web_sm")  # Load the model after downloading

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")  # Load GPT-2 tokenizer
model_gpt2 = GPT2Model.from_pretrained("gpt2")  # Load GPT-2 model for embeddings

# Load GPT-2 decoder model for reconstructing sentences
decoder_model = GPT2LMHeadModel.from_pretrained("gpt2")  # Load GPT-2 model for sentence generation

# Sample clinical sentences to analyze
sentences = [
    "The black patient was prescribed medication.",
    "The white patient was given a different treatment.",
    "The doctor treated the hispanic patient for a respiratory issue.",
    "The nurse assisted the asian patient during recovery."
]

# List of racial terms to check for potential bias
racial_terms = ["black", "white", "hispanic", "asian", "indian", "native", "african"]

# Function to get embeddings from GPT-2
def get_embeddings(sentences):
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt")  # Tokenize the input sentence
        with torch.no_grad():
            outputs = model_gpt2(**inputs)  # Get the model output for the input sentence
            embeddings.append(outputs.last_hidden_state.mean(dim=1))  # Get the mean of the hidden states to represent the sentence
    return torch.cat(embeddings, dim=0)  # Concatenate all sentence embeddings

# Generate positive (biased) and negative (debiased) samples
positive_samples = sentences  # Original biased sentences
negative_samples = [
    sentence.replace(term, "person") for sentence in sentences for term in racial_terms if term in sentence.lower()  # Replace racial terms with neutral terms
]

# Get embeddings for both positive and negative samples
positive_embeddings = get_embeddings(positive_samples)  # Get embeddings for positive samples
negative_embeddings = get_embeddings(negative_samples)  # Get embeddings for negative samples

# Combine positive and negative samples
data_embeddings = torch.cat((positive_embeddings, negative_embeddings), dim=0)  # Combine positive and negative embeddings
labels = torch.cat((torch.ones(len(positive_samples)), torch.zeros(len(negative_samples))), dim=0)  # Create labels for positive (1) and negative (0) samples

# Define a simple contrastive learning model
class ContrastiveModel(nn.Module):
    def __init__(self, embedding_dim):
        super(ContrastiveModel, self).__init__()
        self.fc = nn.Linear(embedding_dim, 128)  # Fully connected layer to project to a lower dimension
        self.projection = nn.Linear(128, 64)  # Projection layer for further reducing dimensionality
    
    def forward(self, x):
        x = torch.relu(self.fc(x))  # Apply ReLU activation function to the fully connected layer
        x = self.projection(x)  # Apply projection layer to further reduce dimensionality
        return x

# Initialize the contrastive model
embedding_dim = data_embeddings.shape[1]  # Get the embedding dimension from the data
contrastive_model = ContrastiveModel(embedding_dim)  # Initialize the contrastive model
criterion = nn.CosineEmbeddingLoss()  # Loss function to measure similarity between positive and negative pairs
optimizer = optim.Adam(contrastive_model.parameters(), lr=0.001)  # Adam optimizer for training the model

# Train the model using contrastive learning
epochs = 10  # Number of training epochs
batch_size = 2  # Batch size for training
for epoch in range(epochs):
    contrastive_model.train()  # Set the model to training mode
    permutation = torch.randperm(data_embeddings.size(0))  # Shuffle the data
    for i in range(0, data_embeddings.size(0), batch_size):
        indices = permutation[i:i + batch_size]  # Get the indices for the current batch
        batch_embeddings = data_embeddings[indices]  # Get the embeddings for the current batch
        batch_labels = labels[indices]  # Get the labels for the current batch
        
        # Create positive and negative pairs for contrastive learning
        if len(batch_embeddings) < 2:
            continue  # Skip if batch size is less than 2
        emb1, emb2 = batch_embeddings[0], batch_embeddings[1]  # Get two embeddings from the batch
        label = 1 if batch_labels[0] == batch_labels[1] else -1  # Determine if the pair is positive or negative
        
        # Forward pass
        output1 = contrastive_model(emb1)  # Get the output for the first embedding
        output2 = contrastive_model(emb2)  # Get the output for the second embedding
        loss = criterion(output1, output2, torch.tensor(label, dtype=torch.float))  # Compute the loss
        
        # Backward pass and optimization
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model parameters
    
    # Print loss for each epoch
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")  # Print the loss for the current epoch

# Save the debiased embeddings
contrastive_model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    debiased_embeddings = contrastive_model(data_embeddings)  # Get the debiased embeddings

print("Debiased embeddings generated.")  # Print confirmation that debiased embeddings have been generated

# Function to reconstruct debiased sentences using the decoder model
def reconstruct_sentences(embeddings, tokenizer, model):
    reconstructed_sentences = []
    for embedding in embeddings:
        # Generate text using the GPT-2 model starting from an embedding
        prompt = "The following text is generated based on an embedding: "  # Use a fixed prompt to initialize generation
        input_ids = tokenizer.encode(prompt, return_tensors="pt")  # Encode the prompt
        outputs = model.generate(input_ids=input_ids, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)  # Generate a sentence
        reconstructed_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Decode the generated output
        reconstructed_sentences.append(reconstructed_sentence)  # Append the reconstructed sentence to the list
    return reconstructed_sentences  # Return the list of reconstructed sentences

# Reconstruct debiased sentences
reconstructed_sentences = reconstruct_sentences(debiased_embeddings, tokenizer, decoder_model)  # Reconstruct sentences using debiased embeddings

# Print reconstructed sentences
print("Reconstructed Sentences:")  # Print header for reconstructed sentences
for sentence in reconstructed_sentences:
    print(sentence)  # Print each reconstructed sentence
